In [1]:
from keras import backend as K
import tensorflow as tf
from math import ceil
import numpy as np
import math
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates

Using TensorFlow backend.


In [2]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
K.set_session(sess)
K.clear_session()


In [3]:
gt = np.load("outputs/predder.npy")
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_to_2']
gt2_proj = gt[1]['predictions_2_to_1']


In [4]:
pred_1 = np.load("outputs/predictions_1_e0.npy")
pred_1_to_2 = np.load("outputs/predictions_1_to_2_e0.npy")
pred_2 = np.load("outputs/predictions_2_e0.npy")
pred_2_proj = np.load("outputs/predictions_2_to_1_e0.npy")


In [5]:
neg_pos_ratio = 3
n_neg_min = 0
alpha = 1

def smooth_L1_loss(y_true, y_pred):
    absolute_loss = tf.abs(y_true - y_pred)
    square_loss = 0.5 * (y_true - y_pred)**2
    l1_loss = tf.where(tf.less(absolute_loss, 1.0), square_loss, absolute_loss - 0.5)
    return tf.reduce_sum(l1_loss, axis=-1)

def log_loss(y_true, y_pred):

    y_pred = tf.maximum(y_pred, 1e-15)
    # Compute the log loss
    log_loss = -tf.reduce_sum(y_true * tf.log(y_pred), axis=-1)
    return log_loss


def compute_loss(y_true, y_pred):
    def gt_rem(pred, gt):
        print("gt_rem")
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)
        predval = tf.shape(pred)
        gtval = tf.shape(gt)
        print("tf.shape(pred)[1]: ", K.eval(predval))
        print("tf.shape(gt)[1]: ", K.eval(gtval))


        val = tf.shape(pred)[1] - tf.shape(gt)[1]
        print("tf.shape(pred)[1]: ", K.eval(tf.shape(pred)[1]))
        gt = tf.slice(gt, [0, 0, 0], [1, tf.shape(pred)[1], 18],name="rem_slice")
        return gt

    def gt_add(pred, gt):
        print("gt_add")
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)
        
        val = tf.shape(pred)[1] - tf.shape(gt)[1]
        print("val: ", K.eval(val))
        ext = tf.slice(gt, [0, 0, 0], [1, 1, 18], name="add_slice")
        
        multiply = tf.constant([1,val,1])
        ext = tf.tile(ext, multiply)
        gt = K.concatenate([ext,gt], axis=1)
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)
        return gt

    def equalalready(gt, pred): return pred

    def make_equal(pred, gt):
        print("pred: ", K.eval(pred).shape)
        print("gt: ", K.eval(gt).shape)


#         equal_tensor = tf.cond(tf.less(tf.shape(pred)[1], tf.shape(gt)[1]), lambda: gt_rem(pred, gt), lambda: gt_add(pred, gt), name="make_equal_cond")
#         equal_tensor = gt_rem(pred, gt)
        equal_tensor = gt_add(pred, gt)
        return equal_tensor


    def matcher(y_true_1,y_pred_1,y_true_2,y_pred_2, bsz):
        pred = 0
        gt = 0

#         for i in range(bsz):
        i = 0
        filterer = tf.where(tf.not_equal(y_true_1[i,:,-4],99))
        filterer_2 = tf.where(tf.not_equal(y_true_2[i,:,-4],99))

        y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer)            
        y_true_new = tf.expand_dims(y_true_new, 0)

        y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
        y_true_2_new = tf.expand_dims(y_true_2_new, 0)

        set1 = tf.cast(y_true_new[i,:,-4],dtype=tf.int32)
        set2 = tf.cast(y_true_2_new[i,:,-4],dtype=tf.int32)

        id_pick = tf.sets.set_intersection(set1[None,:], set2[None, :])
        id_pick = tf.cast(id_pick.values[0],dtype=tf.float64)

        filterer = tf.where(tf.equal(y_true_1[i,:,-4],id_pick))
        filterer_2 = tf.where(tf.equal(y_true_2[i,:,-4],id_pick))

        y_true_new = tf.gather_nd(y_true_1[i,:,:],filterer)            
        y_true_new = tf.expand_dims(y_true_new, 0)

        y_true_2_new = tf.gather_nd(y_true_2[i,:,:],filterer_2)
        y_true_2_new = tf.expand_dims(y_true_2_new, 0)

#             print("y_pred_1:", y_pred_1[i,:,-16:-12].shape)
#             print("y_true_new:", K.eval(y_true_new[i,:,-16:-12]).shape)
#             print("--------------------------------------------------")

        iou_out = tf.py_func(iou, [y_pred_1[i,:,-16:-12],tf.convert_to_tensor(y_true_new[i,:,-16:-12])], tf.float64, name="iou_out")
        bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64, name="bipartite_matches")
        out = tf.gather(y_pred_2[i,:,:], [bipartite_matches], axis=0, name="out")

#             print("iou_out:", K.eval(iou_out).shape)
#             print("bipartite_matches:", K.eval(bipartite_matches).shape)
#             print("out:", K.eval(out).shape)
#             print("--------------------------------------------------")

#             print("tf.shape(out)[1]:", K.eval(tf.shape(out)[1]))
#             print("tf.shape(y_true_2_new)[1]:", K.eval(tf.shape(y_true_2_new)[1]))

        box_comparer = tf.reduce_all(tf.equal(tf.shape(out)[1], tf.shape(y_true_2_new)[1]), name="box_comparer")
        print("box_comparer:", K.eval(box_comparer))
        print("--------------------------------------------------")


        y_true_2_equal = tf.cond(box_comparer, lambda: equalalready(y_true_2_new,out), lambda: make_equal(out, y_true_2_new), name="y_true_cond")
#         y_true_2_equal = tf.identity(y_true_2_equal, name="y_true_2_equal")

        if i != 0:
            pred = K.concatenate([pred,out], axis=-1)
            gt = K.concatenate([gt,y_true_2_equal], axis=0)
        else:
            pred = out
            gt = y_true_2_equal    
        return pred, gt

        
    y_true_1 = y_true[:,:,:18]
    y_pred_1 = y_pred[:,:,:18]
    y_true_2 = y_true[:,:,18:]
    y_pred_2 = y_pred[:,:,18:]

    y_pred, y_true = matcher(y_true_1,y_pred_1,y_true_2,y_pred_2,1)
                
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.float32)

    y_pred1 = y_pred
    t_true1 = y_true

    batch_size = tf.shape(y_pred1)[0]
    n_boxes = tf.shape(t_true1)[1] 

    classification_loss = tf.to_float(log_loss(t_true1[:,:,:-16], y_pred1[:,:,:-16])) # Output shape: (batch_size, n_boxes)
    localization_loss = tf.to_float(smooth_L1_loss(t_true1[:,:,-16:-12], y_pred1[:,:,-16:-12])) # Output shape: (batch_size, n_boxes)

    negatives = t_true1[:,:,0] # Tensor of shape (batch_size, n_boxes)
    positives = tf.to_float(tf.reduce_max(t_true1[:,:,1:-16], axis=-1)) # Tensor of shape (batch_size, n_boxes)
    n_positive = tf.reduce_sum(positives)

    pos_class_loss = tf.reduce_sum(classification_loss * positives, axis=-1) # Tensor of shape (batch_size,)


    neg_class_loss_all = classification_loss * negatives # Tensor of shape (batch_size, n_boxes)
    n_neg_losses = tf.count_nonzero(neg_class_loss_all, dtype=tf.int32) # The number of non-zero loss entries in `neg_class_loss_all`
    n_negative_keep = tf.minimum(tf.maximum(neg_pos_ratio * tf.to_int32(n_positive), n_neg_min), n_neg_losses)

    def f1():
        return tf.zeros([batch_size])
    def f2():

        neg_class_loss_all_1D = tf.reshape(neg_class_loss_all, [-1]) # Tensor of shape (batch_size * n_boxes,)
        values, indices = tf.nn.top_k(neg_class_loss_all_1D,
                                      k=n_negative_keep,
                                      sorted=False) # We don't need them sorted.

        negatives_keep = tf.scatter_nd(indices=tf.expand_dims(indices, axis=1),
                                       updates=tf.ones_like(indices, dtype=tf.int32),
                                       shape=tf.shape(neg_class_loss_all_1D)) # Tensor of shape (batch_size * n_boxes,)
        negatives_keep = tf.to_float(tf.reshape(negatives_keep, [batch_size, n_boxes])) # Tensor of shape (batch_size, n_boxes)
        # ...and use it to keep only those boxes and mask all other classification losses
        neg_class_loss = tf.reduce_sum(classification_loss * negatives_keep, axis=-1) # Tensor of shape (batch_size,)
        return neg_class_loss

    neg_class_loss = tf.cond(tf.equal(n_neg_losses, tf.constant(0)), f1, f2)

    class_loss = pos_class_loss + neg_class_loss # Tensor of shape (batch_size,)

    loc_loss = tf.reduce_sum(localization_loss * positives, axis=-1) # Tensor of shape (batch_size,)

    # 4: Compute the total loss.

    total_loss = (class_loss + alpha * loc_loss) / tf.maximum(1.0, n_positive) # In case `n_positive == 0`
    total_loss = total_loss * tf.to_float(batch_size)
    total_loss.set_shape((None,))
    return total_loss

In [6]:
gt1_proj = tf.convert_to_tensor(gt1_proj)
pred_1_to_2 = tf.convert_to_tensor(pred_1_to_2)
loss = compute_loss(gt1_proj,pred_1_to_2)
print("loss:",K.eval(loss))

box_comparer: False
--------------------------------------------------
pred:  (1, 17292, 18)
gt:  (1, 8, 18)
gt_add
pred:  (1, 17292, 18)
gt:  (1, 8, 18)


ValueError: Operation 'y_true_cond/sub' has been marked as not fetchable.

In [7]:
print(gt1_proj.shape)
multiply = tf.constant([1,2,1])
f = tf.tile(gt1_proj, multiply)
K.eval(f).shape

(4, 17292, 36)


(4, 34584, 36)

In [10]:
ext = tf.slice(gt1_proj, [0, 0, 0], [1, 1, 36])
ext.shape

TensorShape([Dimension(1), Dimension(1), Dimension(36)])